Inspirations:

https://blog.keras.io/building-autoencoders-in-keras.html
https://keras.io/examples/vision/autoencoder/



---



We're using MNIST digits, and we're discarding the labels (since we're only interested in encoding/decoding the input images).



In [ ]:
from keras.datasets import mnist
import numpy as np

(x_train, _), (x_test, y_test) = mnist.load_data()

x_train = x_train.astype('float32') / 255.
x_test = x_test.astype('float32') / 255.
x_train = np.reshape(x_train, (len(x_train), 28, 28, 1))
x_test = np.reshape(x_test, (len(x_test), 28, 28, 1))

noise_factor = 0.5
x_train_noisy = x_train + noise_factor * np.random.normal(loc=0.0, scale=1.0, size=x_train.shape)
x_test_noisy = x_test + noise_factor * np.random.normal(loc=0.0, scale=1.0, size=x_test.shape)

x_train_noisy = np.clip(x_train_noisy, 0., 1.)
x_test_noisy = np.clip(x_test_noisy, 0., 1.)



---



In [ ]:
import keras
from keras import layers

input_img = keras.Input(shape=(28, 28, 1))

In [ ]:
x = layers.Conv2D(16, (3, 3), activation='relu', padding='same')(input_img)
x = layers.MaxPooling2D((2, 2), padding='same')(x)
x = layers.Conv2D(8, (3, 3), activation='relu', padding='same')(x)
x = layers.MaxPooling2D((2, 2), padding='same')(x)
x = layers.Conv2D(8, (3, 3), activation='relu', padding='same')(x)
encoded = layers.MaxPooling2D((2, 2), padding='same')(x)

# at this point the representation is (4, 4, 8) i.e. 128-dimensional

x = layers.Conv2D(8, (3, 3), activation='relu', padding='same')(encoded)
x = layers.UpSampling2D((2, 2))(x)
x = layers.Conv2D(8, (3, 3), activation='relu', padding='same')(x)
x = layers.UpSampling2D((2, 2))(x)
x = layers.Conv2D(16, (3, 3), activation='relu')(x)
x = layers.UpSampling2D((2, 2))(x)
decoded = layers.Conv2D(1, (3, 3), activation='sigmoid', padding='same')(x)

In [ ]:
autoencoder = keras.Model(input_img, decoded)
encoder = keras.Model(input_img, encoded)

As well as the decoder model:

In [ ]:
autoencoder.compile(optimizer='adam', loss='binary_crossentropy')

In [ ]:
autoencoder.fit(x_train_noisy, x_train,
                epochs=50,
                batch_size=256,
                shuffle=True,
                validation_data=(x_test_noisy , x_test))

 We can visualize the reconstructed inputs and the encoded representations.

In [ ]:
# Encode and decode some digits
# Note that we take them from the *test* set
encoded_imgs = encoder.predict(x_test_noisy)
decoded_imgs = autoencoder.predict(x_test_noisy)

In [ ]:
# Use Matplotlib (don't ask)
import matplotlib.pyplot as plt

n = 10  # How many digits we will display
plt.figure(figsize=(20, 4))
for i in range(n):
    # Display original
    ax = plt.subplot(2, n, i + 1)
    plt.imshow(x_test_noisy[i].reshape(28, 28))
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)

    # Display reconstruction
    ax = plt.subplot(2, n, i + 1 + n)
    plt.imshow(decoded_imgs[i].reshape(28, 28))
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
plt.show()



---



In [ ]:
from sklearn.decomposition import PCA
# Użyj PCA do wizualizacji klastrów
pca = PCA(n_components=2)  # Redukcja wymiarów za pomocą PCA do 2 komponentów
encoded_imgs_pca=encoded_imgs.reshape((-1, 4*4*8))
encoded_imgs_2d = pca.fit_transform(encoded_imgs_pca)  # Przekształcenie zakodowanych wektorów do 2D

In [ ]:
from sklearn.cluster import KMeans
# Użyj KMeans do analizy klastrów
kmeans = KMeans(n_clusters=10, random_state=0)  # Algorytm KMeans z 10 klastrami
clusters = kmeans.fit_predict(encoded_imgs_pca)     # Dopasowanie modelu KMeans i przypisanie klastrów

In [ ]:
# Wizualizacja klastrów
plt.figure(figsize=(10, 8))  # Ustawienie rozmiaru wykresu
plt.scatter(encoded_imgs_2d[:, 0], encoded_imgs_2d[:, 1], c=clusters, cmap='tab10')  # Wykres rozproszenia z kolorami odpowiadającymi klastrom
plt.colorbar()  # Dodanie paska kolorów
plt.show()  # Wyświetlenie wykresu

In [ ]:
# Mapowanie klastrów na oryginalne etykiety
def map_clusters_to_labels(clusters, labels):
    label_map = {}  # Słownik do mapowania klastrów na etykiety
    for cluster in np.unique(clusters):  # Iteracja po unikalnych klastrach
        indices = np.where(clusters == cluster)  # Znalezienie indeksów próbek należących do danego klastra
        true_labels = labels[indices]  # Pobranie prawdziwych etykiet dla tych próbek
        most_common_label = np.bincount(true_labels).argmax()  # Znalezienie najczęściej występującej etykiety
        label_map[cluster] = most_common_label  # Przypisanie najczęstszej etykiety do klastra
    print(f"Przypisanie najczęstszej etykiety do klastra {label_map}")
    return np.vectorize(label_map.get)(clusters)  # Mapowanie klastrów na etykiety

In [ ]:
from sklearn.metrics import accuracy_score
predicted_labels = map_clusters_to_labels(clusters, y_test)  # Mapowanie klastrów na przewidywane etykiety
accuracy = accuracy_score(y_test, predicted_labels)  # Obliczenie dokładności klastrowania
print(f"accuracy={accuracy}")

In [ ]:
# Wyświetlenie macierzy konfuzji
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, predicted_labels)  # Macierz konfuzji dla prawdziwych i przewidywanych etykiet
plt.figure(figsize=(10, 8))  # Ustawienie rozmiaru wykresu
import seaborn as sns
sns.heatmap(cm, annot=True, fmt="d", cmap='Blues')  # Wykres macierzy konfuzji
plt.xlabel('Predicted')  # Etykieta osi x
plt.ylabel('True')  # Etykieta osi y
plt.show()  # Wyświetlenie wykresu